In [1]:
from random import randint
from time import sleep
import pandas as pd
import csv
from textblob import TextBlob  
import requests
from bs4 import BeautifulSoup as soup 
from selenium import webdriver

# getting links

In [1]:
def get_link(url):
    try:
        driver = webdriver.Firefox()
        driver.get(url)
        select = driver.find_element_by_id("language")
        select.click()
        select.find_element_by_xpath("//option[@value='all']").click()
        driver.find_element_by_xpath("//input[@value='Submit' or @type='Submit']").click()
        link = driver.current_url
        driver.quit()
    except:
        driver.quit()
    return(link)

In [4]:
html =requests.get('https://www.booking.com/reviews/ma/city/marrakech.en-gb.html?aid=356980;label=gog235jc-1DCCoojAE4mANIM1gDaIwBiAEBmAEJuAEXyAEP2AED6AEB-AEDiAIBqAIDuAKw8MKNBsACAdICJGEwNjg1ZjE4LTVkZGEtNDRmZC04MzhlLWNhZTk1ZmJkOWMzNtgCBOACAQ;sid=30f3f38f6591fa674ec593b0aba5b7b5;keep_landing=1&offset=390&')
html.status_code

200

In [5]:
bsobj = soup(html.content,'lxml')


In [ ]:
filelinks.close()

# Saving Links

In [ ]:
import csv
filelinks=open('linksMarrakech.csv','w',encoding='utf-8')
writer=csv.writer(filelinks)
writer.writerow(['links'])


for review in bsobj.findAll('li',{'class':'rlp-main-hotel-review__review_link'}):
    links = []
    a = review.a['href']
    a = 'https://www.booking.com'+ a
    a= get_link(a)
    links.append(a)
    writer.writerow(links)
while(bsobj.find('a',{'class':'rlp-main-pagination__btn-txt--next'})):
    next_lk=bsobj.find('a',{'class':'rlp-main-pagination__btn-txt--next'})['href']
    next_lk='https://www.booking.com'+next_lk
    html =requests.get(next_lk)
    bsobj = soup(html.content,'lxml')
    for review in bsobj.findAll('li',{'class':'rlp-main-hotel-review__review_link'}):
        links = []
        a = review.a['href']
        a = 'https://www.booking.com'+ a
        a= get_link(a)
        links.append(a)
        writer.writerow(links)

filelinks.close()

In [4]:
def writeReview(date,ville,name_hotel,rev,typerev,rate,username,usercountry):
    rev =(' '.join(rev.splitlines()))
    rev=rev.replace('.',' ')
    tx=TextBlob(rev)
    sleep(randint(1,2))
    if(tx.detect_language() != 'en'):
        tr = tx.translate(to = 'en')
        rev=str(tr)
    row = [ name_hotel ,ville, rev,typerev, date, rate, username, usercountry]
    writer.writerow(row)


In [5]:
def check_internet():
    try:
        requests.get('https://google.com').ok
        return True
    except :
        return False
print(check_internet())

True


# Saving Reviews

In [ ]:
hotels = []
ville='Agadir'
#open csv file 
file = open('BookingReviewsAgadirV1.csv', 'w', encoding='UTF-8')
writer = csv.writer(file)
#set file header
header = ["hotel_name","ville", "review","type_rev","stay_date","reviewer_rate","reviewer_name","reviewer_country"]
writer.writerow(header)
links=pd.read_csv('linksAgadir.csv',encoding='utf-8')['links']
for link in links:
    try:
        link=link.strip()
        headers = {'User-Agent':'Mozilla/5.0 '}
        html2 = requests.get(link.format(i for i in range(5,25,5)),headers=headers)
        sleep(randint(1,5))
        bsobj2 = soup(html2.content,'lxml')
        name_hotel=bsobj2.find('a',{'class':'standalone_header_hotel_link'}).text
        if name_hotel not in hotels:
            hotels.append(name_hotel)
        for r in bsobj2.findAll('li',{'class':'review_item clearfix'}):
            try:
                rate=r.find('span',{'class':'review-score-badge'}).text.strip()
                username=r.find('p',{'class':'reviewer_name'}).span.text.strip()
                usercountry=r.find('span',{'itemprop':'nationality'}).span.text.strip()
                revpos = r.find('p', {'class' : 'review_pos'})
                revneg = r.find('p', {'class' : 'review_neg'})
                date = r.find('p', {'class':'review_staydate'}).text.strip()
                date = date[10:]
                if revpos:
                    rev = revpos.span.text.strip() 
                    writeReview(date,ville,name_hotel,rev,"pos",rate,username,usercountry)
                if revneg:
                    rev = revneg.span.text.strip()
                    writeReview(date,ville,name_hotel,rev,"neg",rate,username,usercountry)
            except Exception as e:
                print(e,e.args)
                while(not check_internet()):
                    sleep(20)
                continue
        next_page=bsobj2.find('p',{'class':'page_link review_next_page'})
        while(next_page.find('a',{'id':'review_next_page_link'})):
            try:
                link_next=next_page.a['href']
                link_next='https://www.booking.com'+ link_next
                link_next=link_next.strip()
                html2 = requests.get(link_next.format(i for i in range(5,25,5)),headers=headers)
                sleep(1)
                bsobj2 = soup(html2.content,'lxml')
                name_hotel=bsobj2.find('a',{'class':'standalone_header_hotel_link'}).text
                if name_hotel not in hotels:
                    hotels.append(name_hotel)
                for r in bsobj2.findAll('li',{'class':'review_item clearfix'}):
                    try:
                        rate=r.find('span',{'class':'review-score-badge'}).text.strip()
                        username=r.find('p',{'class':'reviewer_name'}).span.text.strip()
                        usercountry=r.find('span',{'itemprop':'nationality'}).span.text.strip()
                        revpos = r.find('p', {'class' : 'review_pos'})
                        revneg = r.find('p', {'class' : 'review_neg'})
                        date = r.find('p', {'class':'review_staydate'}).text.strip()
                        date = date[10:]
                        if revpos:
                            rev = revpos.span.text.strip() 
                            writeReview(date,ville,name_hotel,rev,"pos",rate,username,usercountry)
                        if revneg:
                            rev = revneg.span.text.strip()
                            writeReview(date,ville,name_hotel,rev,"neg",rate,username,usercountry)
                    except Exception as e:
                        print(e)
                        while(not check_internet()):
                            sleep(20)
                        continue
                next_page=bsobj2.find('p',{'class':'page_link review_next_page'})
            except Exception as e:
                print(e)
                while(not check_internet()):
                    sleep(20)
                continue 
    except Exception as e:
        print(e)
        while(not check_internet()):
            sleep(20)
        continue
file.close()

Must provide a string with at least 3 characters. ('Must provide a string with at least 3 characters.',)
Must provide a string with at least 3 characters. ('Must provide a string with at least 3 characters.',)
'NoneType' object has no attribute 'text' ("'NoneType' object has no attribute 'text'",)
HTTP Error 400: Bad Request
'NoneType' object has no attribute 'text'
Translation API returned the input string unchanged.
Must provide a string with at least 3 characters. ('Must provide a string with at least 3 characters.',)
Translation API returned the input string unchanged. ('Translation API returned the input string unchanged.',)
<urlopen error [WinError 10060] Une tentative de connexion a échoué car le parti connecté n’a pas répondu convenablement au-delà d’une certaine durée ou une connexion établie a échoué car l’hôte de connexion n’a pas répondu>
<urlopen error [Errno 11001] getaddrinfo failed> (gaierror(11001, 'getaddrinfo failed'),)
HTTP Error 400: Bad Request ()
Must provide a s

In [1]:
import pandas as pd
df=pd.DataFrame(data=hotels,columns=["hotels"])
df.to_csv('hotelsV3.csv',encoding='UTF-8')